In [1]:
import numpy as np
import re

## Day 1

In [2]:
with open('day1Input.txt') as f:
    data = f.readlines()

numbers = []
for string in data:
    string = string.split("\n")[0]
    # find the first number
    first = re.search('(\d)', string)

    # Find the last (would be the same as first if only one number)
    last = re.search('(\d)', string[::-1])

    numbers.append(first.group(1)+last.group(1))
    
# Make an array of integers and sum
numbers = np.asarray(numbers, dtype=int)
print(numbers.sum())

55621


Now we try to account also for numbers that are written down 

In [3]:
with open('day1Input.txt') as f:
    data = f.readlines()


# Make a dictionary of all the numbers
numConv = {'one':'1', 'two':'2', 'three':'3', 'four':'4', 'five':'5',
           'six':'6', 'seven':'7', 'eight':'8', 'nine':'9'}




def ReplaceNumbers(string, mapping=numConv):
    '''
    Replaces written numbers with the numerics
    
    INPUT >>>  string = String to check over
               map = Number conversion mapping
              
    OUTPUT >>> string [with all numeric numbers]
    '''
    for word, digit in mapping.items():
        string = re.sub(word, word[0]+digit+word[-1], string, count=0, flags=re.IGNORECASE)
    return string

numbers = []
for string in data:
    string=string.split('\n')[0]
    # convert the numbers to digits
    string = ReplaceNumbers(string)

    # find the first number
    first = re.search('(\d)', string)

    # Find the last (would be the same as first if only one number)
    last = re.search('(\d)', string[::-1])
    numbers.append(first.group(1)+last.group(1))
    
# Make an array of integers and sum
numbers = np.asarray(numbers, dtype=int)


## Day 2

In [33]:
# Dictionary of actual max ball numbers
maxNums = {'red':12,
           'green':13,
           'blue':14}

with open('day2Input.txt') as f:
    data = f.readlines()

# Keep track of legit games
goodGames=[]
for game in data:
    # Find the maximum number of balls shown of each colour in the given game
    redNum = max([int(re.sub(' red', '', string)) for string in re.findall(r'(\d+ red)', game)])
    greenNum = max([int(re.sub(' green', '', string)) for string in re.findall(r'(\d+ green)', game)])
    blueNum = max([int(re.sub(' blue', '', string)) for string in re.findall(r'(\d+ blue)', game)])

    # If all of these maximums are less than the max allowed then the game is legit and store the game id
    if (redNum<=maxNums['red'])&(greenNum<=maxNums['green'])&(blueNum<=maxNums['blue']):
        goodGames.append(int(re.search('(Game \d+)', game).group(1).replace('Game ', '')))

print(f'{len(goodGames)} possible legit games with summed ids to {sum(goodGames)}')


46 possible legit games with summed ids to 2283


Now we find the fewest number of cubes of each colour that could have made the game possible

In [34]:
with open('day2Input.txt') as f:
    data = f.readlines()

cubePower=[]
for game in data:
    # Find the maximum number of balls shown of each colour in the given game
    # This max number is also the fewest needed of that colour
    redNum = max([int(re.sub(' red', '', string)) for string in re.findall(r'(\d+ red)', game)])
    greenNum = max([int(re.sub(' green', '', string)) for string in re.findall(r'(\d+ green)', game)])
    blueNum = max([int(re.sub(' blue', '', string)) for string in re.findall(r'(\d+ blue)', game)])

    # Append the product of these values
    cubePower.append(redNum*greenNum*blueNum)

print(f'Sum of cube power is {sum(cubePower)}')
    

Sum of cube power is 78669


## Day 3

In [73]:
def FindSymbols(string, pattern=r'[^\d.]'):
    '''
    Finds special symbols (not digits or periods)
    in a string.

    Returns True if symbol found
    '''
    matches = re.finditer(pattern, string)
    if len([m.group() for m in matches])!=0:
        return True
    else:
        return False


# Read in the data without the line breaks at the end
data = np.array([re.sub('\n', '', line) for line in open('day3Input.txt')])
# data = np.array([re.sub('\n', '', line) for line in open('day3Test')])

nums=[]
# Loop through every line and find adjacent numbers
for i, line in enumerate(data):
    # Find numbers in current line
    numbers = re.finditer(r'(\d+)', line)
    # if at first line only look to line 2
    if i==0:
        # look at the indices surrounding each number
        for N in numbers:
            # Set the start and end points
            if N.start()==0: start=N.start()
            else: start=N.start()-1
            if N.end()==len(line)-1: end=N.end()
            else: end=N.end()+1

            # If found in next to number, add to list and skip to next number
            if FindSymbols(line[start:end]):
                nums.append(N.group())
                continue

            elif FindSymbols(data[i+1][start:end]):
                nums.append(N.group())
    
    # If at final line only look at line before and current one 
    elif i==len(data)-1:
        # look at the indices surrounding each number
        for N in numbers:
            # Set the start and end points
            if N.start()==0: start=N.start()
            else: start=N.start()-1
            if N.end()==len(line)-1: end=N.end()
            else: end=N.end()+1

            # If found in next to number, add to list and skip to next number
            if FindSymbols(data[i-1][start:end]):
                nums.append(N.group())
                continue
            
            elif FindSymbols(line[start:end]):
                nums.append(N.group())
                
            
    # Else look at the line above and below
    else:
        # look at the indices surrounding each number
        for N in numbers:
            # Set the start and end points
            if N.start()==0: start=N.start()
            else: start=N.start()-1
            if N.end()==len(line)-1: end=N.end()
            else: end=N.end()+1

            # If found in next to number, add to list and skip to next number
            if FindSymbols(data[i-1][start:end]):
                nums.append(N.group())
                continue

            elif FindSymbols(line[start:end]):
                nums.append(N.group())
                continue
            
            elif FindSymbols(data[i+1][start:end]):
                nums.append(N.group())
                

nums = np.asarray(nums, dtype=int)

print(f'Sum of all part numbers in engine scheme is {sum(nums)}')
print(nums)

Sum of all part numbers in engine scheme is 522726
[501 168 202 ... 855 261  42]


Now we search for "gears" which are exactly 2 numbers around an *

We then compute the gear "ratio" (product of numbers)

Finally we sum these gear ratios

In [167]:
def CalcDist(centre, pos):
    '''
    Computes the distance between numbers and an astreix
    '''
    return np.sqrt(np.sum((pos-centre)**2, axis=1))


# Read in the data without the line breaks at the end
data = np.array([re.sub('\n', '', line) for line in open('day3Input.txt')])
# data = np.array([re.sub('\n', '', line) for line in open('day3Test')])

nums=[]; startPos=[]; endPos=[]
astrx=[]
for i, line in enumerate(data):
    # Find centers
    astrx.extend([[a.start(),i] for a in re.finditer(r'(\*)', line)])
    
    # Find numbers and coords
    nums.extend([int(N.group()) for N in re.finditer(r'(\d+)', line)])
    startPos.extend([N.start(),i] for N in re.finditer(r'(\d+)', line))
    endPos.extend([N.end()-1,i] for N in re.finditer(r'(\d+)', line))

# Make array for ease
nums=np.asarray(nums).flatten()
astrx = np.asarray(astrx)
startPos = np.asarray(startPos)
endPos = np.asarray(endPos)

gr=[]
for a in astrx:
    # Calculate distance to start of number and end
    startDist=CalcDist(a, startPos)
    endDist=CalcDist(a, endPos)

    # Find where numbers are close to astrix 
    numIndex = np.where((startDist<2)|(endDist<2))
    gear = nums[numIndex]

    # If exactly two numbers next to astrix find gear ratio
    if len(gear)==2:
        gr.append(gear[0]*gear[1])

print(f'Sum of gear ratios is {sum(gr)}')

Sum of gear ratios is 81721933


## Day 4

In [224]:
# Load the data 
data = np.array([re.sub('\n', '', line) for line in open('day4Input.txt')])

pointsPerCard=[]
for line in data:
    # Remove unnecessary characters and split winning nums and card nums
    line = re.sub(r'(Card\s+\d+\:\s+)', '', line)
    line = re.sub(r'(\s+\|\s+)', '|', line)
    line = re.sub(r'(\s+)', ' ', line)
    
    # Split into two arrays
    winNums = np.asarray(line.split('|')[0].split(' '), dtype=int); myNums = np.asarray(line.split('|')[1].split(' '), dtype=int)
    
    # find how many my nums in winning nums
    number = sum(np.isin(myNums, winNums))

    # points on the card is then 1 X 2^(number of match)
    pointsPerCard.append(2**number)
    
print(f'Total number of points {sum(pointsPerCard)}')

Total number of points 35641


In [210]:
myNums

array(['0'], dtype='<U1')